In [5]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier



In [7]:
file_path = r"/Users/sa8/Desktop/The-CyberChase/CyberCrimePredicitionModel/CleanCSVforModel/merged_df_clean.csv"
train_data = pd.read_csv(file_path)
